In [168]:
# import the datetime class
from datetime import datetime

# Load the data from a CSV file using a semicolon as the delimiter character
renting_costs_df = pd.read_csv('renting_costs_austria_2004_2021.csv', delimiter=';')
# Print the first 5 rows of the dataframe
renting_costs_df.head()

,Location,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Burgenland,"4,06","3,97","3,91","4,21","4,43","4,41","4,64","4,78","4,86","5,06","5,36","5,51","5,82","5,93","6,02","6,43","6,33"
1,Kärnten,"4,55","4,61","4,76","4,84","4,88","4,93","5,13","5,27","5,38","5,56","5,71","5,89","6,03","6,15","6,34","6,49","6,43"
2,Niederösterreich,"4,77","4,82","4,86","5,05","5,24","5,42","5,59","5,89","6,04","6,24","6,30","6,53","6,81","6,92","7,09","7,43","7,37"
3,Oberösterreich,"5,15","5,25","5,24","5,46","5,69","5,75","5,88","6,08","6,32","6,48","6,67","6,86","7,15","7,36","7,50","7,68","7,65"
4,Salzburg,"6,49","6,64","6,97","6,95","7,03","7,12","7,41","7,93","8,06","8,36","8,66","9,04","9,17","9,23","9,53","9,90","10,06"


In [169]:
def tidy_dataframe(df):

    # Select the first column as the fixed column and the remaining columns as the data columns
    fixed_col = renting_costs_df.iloc[:, 0]
    # duplicate it 17 times
    fixed_col = pd.concat([fixed_col]*17, ignore_index=True)

    data_cols = renting_costs_df.iloc[:, 1:]

    # Use the Pandas "melt" function to create a new dataframe where the data columns are duplicated in the "year" and "value" columns
    df_melted = pd.melt(data_cols, value_name='Renting_Costs', var_name='Year')

    # Combine the fixed column and the melted dataframe to create the final dataframe
    df_final = pd.concat([fixed_col, df_melted], axis=1)

    # cast renting cost to float
    df_final["Renting_Costs"] = df_final["Renting_Costs"].str.replace(',', ".").astype(float)

    # cast ints of year to be a date 
    df_final["Year"] = pd.to_datetime(df_final["Year"], format="%Y")

    print(df_final.info())

    # save it as csv
    df_final.to_csv('tidy_dataframes/renting_costs_2005_2021.csv', sep=';')

    return df_final

In [170]:
renting_costs_df_tidy = tidy_dataframe(renting_costs_df)

AttributeError: 'DatetimeProperties' object has no attribute 'end_of_year'

In [159]:
renting_costs_df_tidy.head(18)

,Location,Year,Renting_Costs
0,Burgenland,2005,4.06
1,Kärnten,2005,4.55
2,Niederösterreich,2005,4.77
3,Oberösterreich,2005,5.15
4,Salzburg,2005,6.49
5,Steiermark,2005,5.12
6,Tirol,2005,6.27
7,Voralberg,2005,6.21
8,Wien,2005,5.34
9,Burgenland,2006,3.97


In [164]:

for value in renting_costs_df_tidy['Year']:
    print(type(value))
    

KeyError: 'year'